In [1]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy

First, pulling in modgroups. Contains ~296k unique names and mod classes. Will append this value to the final data

In [2]:
mod_groups = pd.read_csv('/Users/robertpagano/Desktop/AOC final graphs/Mod_class_mccain.csv')

In [3]:
with open('/Users/robertpagano/metis_data/project_5/final_raw_mccain_df.pickle', 'rb') as f:
    df_mccain = pickle.load(f)

In [4]:
df_mccain.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,...,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote
0,1109131638112432129,113817127,KDW,WYATT251,Fri Mar 22 16:36:03 +0000 2019,RT @thedilleyshow: Donald J. Trump is still Pr...,0,107,474,False,...,299.0,8815.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,1109131636262744066,4839872717,Lou Ella Howard,ellahoward271,Fri Mar 22 16:36:03 +0000 2019,RT @LindseyGrahamSC: Great read. Spot on.\n\n...,0,65,3601,False,...,181.0,756904.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,1109131635742564352,881906625799266304,God Is LOVE,janetlchatham,Fri Mar 22 16:36:03 +0000 2019,RT @LouDobbs: #TrumpTrain- @RealDonaldTrump is...,0,158,153,False,...,366.0,1901004.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
3,1109131633603440640,242425184,Dominika Markova,dominikam42,Fri Mar 22 16:36:02 +0000 2019,RT @votevets: 1. John McCain was a war hero.\n...,0,17392,198,False,...,62408.0,158248.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,1109131633293037574,17200919,Michael Little,uzererr,Fri Mar 22 16:36:02 +0000 2019,RT @brithume: President Trump’s attacks on the...,0,1521,74,False,...,9022.0,996715.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [5]:
df_mccain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1037841 entries, 0 to 4071
Data columns (total 28 columns):
tweet_id                   1037841 non-null int64
user_id                    1037841 non-null int64
display_name               1037841 non-null object
screen_name                1037841 non-null object
tweet_date                 1037841 non-null object
main_text                  1037841 non-null object
fav_count                  1037841 non-null int64
rt_count                   1037841 non-null int64
followers_count            1037841 non-null int64
is_verified                1037841 non-null bool
reply_to_tweet_id          70818 non-null float64
reply_to_user_id           81418 non-null float64
reply_to_screen_name       81418 non-null object
retweet_text               895858 non-null object
retweet_id                 895858 non-null float64
retweet_user_id            895858 non-null float64
retweet_screen_name        895858 non-null object
retweet_is_verified        895858 no

In [6]:
df_mccain['screen_name'].nunique()

341517

In [7]:
mod_groups['Id'].nunique()

288320

Not sure why, but we must have cut out around 150k names from the network data. Should probably figure out who that is before continuing, I don't want to not include important accounts

In [8]:
mod_groups['modularity_class'].nunique()

17

In [9]:
mod_groups.groupby(['modularity_class']).count()[['Id']]

,Id
modularity_class,
0,53044
1,62601
2,8
3,9287
4,15843
5,6410
6,8406
7,6495
8,16818


SI can ignore:

- 2
- 16

 
So we need to find the top accounts in each group, do NLP analysis of each group (topic modeling / sentiment analysis)


Next, to check - append community to each account. Then make sure the top x accounts have one of these communties

In [10]:
mod_groups.head()

,Id,Label,timeset,modularity_class,Weighted Degree
0,WYATT251,WYATT251,NaN,9,2
1,thedilleyshow,thedilleyshow,NaN,9,461
2,ellahoward271,ellahoward271,NaN,9,2
3,LindseyGrahamSC,LindseyGrahamSC,NaN,9,41
4,janetlchatham,janetlchatham,NaN,9,3


In [11]:
mod_groups.drop(columns=['Id', 'timeset'], inplace=True)
mod_groups.head()

,Label,modularity_class,Weighted Degree
0,WYATT251,9,2
1,thedilleyshow,9,461
2,ellahoward271,9,2
3,LindseyGrahamSC,9,41
4,janetlchatham,9,3


# TO DO

1. Append community to DF
2. Check top accounts are a part of community (and if they are, look back to see who we lost)
3. Topic modeling with LSA / NMF. Get top words each topic, etc.
4. New Jupyter Notebook - sentiment analysis on each topic by community

In [12]:
df_mccain = pd.merge(df_mccain, mod_groups, how='left', left_on='screen_name', right_on='Label')
df_mccain.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,...,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree
0,1109131638112432129,113817127,KDW,WYATT251,Fri Mar 22 16:36:03 +0000 2019,RT @thedilleyshow: Donald J. Trump is still Pr...,0,107,474,False,...,NaN,NaN,NaN,NaN,NaN,1,0,WYATT251,9.0,2.0
1,1109131636262744066,4839872717,Lou Ella Howard,ellahoward271,Fri Mar 22 16:36:03 +0000 2019,RT @LindseyGrahamSC: Great read. Spot on.\n\n...,0,65,3601,False,...,NaN,NaN,NaN,NaN,NaN,1,0,ellahoward271,9.0,2.0
2,1109131635742564352,881906625799266304,God Is LOVE,janetlchatham,Fri Mar 22 16:36:03 +0000 2019,RT @LouDobbs: #TrumpTrain- @RealDonaldTrump is...,0,158,153,False,...,NaN,NaN,NaN,NaN,NaN,1,0,janetlchatham,9.0,3.0
3,1109131633603440640,242425184,Dominika Markova,dominikam42,Fri Mar 22 16:36:02 +0000 2019,RT @votevets: 1. John McCain was a war hero.\n...,0,17392,198,False,...,NaN,NaN,NaN,NaN,NaN,1,0,dominikam42,3.0,2.0
4,1109131633293037574,17200919,Michael Little,uzererr,Fri Mar 22 16:36:02 +0000 2019,RT @brithume: President Trump’s attacks on the...,0,1521,74,False,...,NaN,NaN,NaN,NaN,NaN,1,0,uzererr,4.0,4.0


In [13]:
pd.set_option('display.max_columns', 500)

In [14]:
df_mccain.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree
0,1109131638112432129,113817127,KDW,WYATT251,Fri Mar 22 16:36:03 +0000 2019,RT @thedilleyshow: Donald J. Trump is still Pr...,0,107,474,False,NaN,NaN,None,Donald J. Trump is still President.\n\nHillary...,1.109098e+18,1.085581e+18,thedilleyshow,False,299.0,8815.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,WYATT251,9.0,2.0
1,1109131636262744066,4839872717,Lou Ella Howard,ellahoward271,Fri Mar 22 16:36:03 +0000 2019,RT @LindseyGrahamSC: Great read. Spot on.\n\n...,0,65,3601,False,NaN,NaN,None,Great read. Spot on.\n\nMuch more could be sa...,1.109117e+18,4.328953e+08,LindseyGrahamSC,True,181.0,756904.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,ellahoward271,9.0,2.0
2,1109131635742564352,881906625799266304,God Is LOVE,janetlchatham,Fri Mar 22 16:36:03 +0000 2019,RT @LouDobbs: #TrumpTrain- @RealDonaldTrump is...,0,158,153,False,NaN,NaN,None,#TrumpTrain- @RealDonaldTrump is speaking out ...,1.109129e+18,2.648717e+07,LouDobbs,True,366.0,1901004.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,janetlchatham,9.0,3.0
3,1109131633603440640,242425184,Dominika Markova,dominikam42,Fri Mar 22 16:36:02 +0000 2019,RT @votevets: 1. John McCain was a war hero.\n...,0,17392,198,False,NaN,NaN,None,1. John McCain was a war hero.\n\n2. John McCa...,1.108447e+18,1.651334e+07,votevets,True,62408.0,158248.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,dominikam42,3.0,2.0
4,1109131633293037574,17200919,Michael Little,uzererr,Fri Mar 22 16:36:02 +0000 2019,RT @brithume: President Trump’s attacks on the...,0,1521,74,False,NaN,NaN,None,President Trump’s attacks on the late John McC...,1.109097e+18,1.120478e+08,brithume,True,9022.0,996715.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,uzererr,4.0,4.0


Now I want to put a counter for each screen name as a row

In [15]:
df_mccain['screenname_freq'] = df_mccain.groupby('screen_name')['screen_name'].transform('count')

In [16]:
df_mccain.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq
0,1109131638112432129,113817127,KDW,WYATT251,Fri Mar 22 16:36:03 +0000 2019,RT @thedilleyshow: Donald J. Trump is still Pr...,0,107,474,False,NaN,NaN,None,Donald J. Trump is still President.\n\nHillary...,1.109098e+18,1.085581e+18,thedilleyshow,False,299.0,8815.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,WYATT251,9.0,2.0,2
1,1109131636262744066,4839872717,Lou Ella Howard,ellahoward271,Fri Mar 22 16:36:03 +0000 2019,RT @LindseyGrahamSC: Great read. Spot on.\n\n...,0,65,3601,False,NaN,NaN,None,Great read. Spot on.\n\nMuch more could be sa...,1.109117e+18,4.328953e+08,LindseyGrahamSC,True,181.0,756904.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,ellahoward271,9.0,2.0,6
2,1109131635742564352,881906625799266304,God Is LOVE,janetlchatham,Fri Mar 22 16:36:03 +0000 2019,RT @LouDobbs: #TrumpTrain- @RealDonaldTrump is...,0,158,153,False,NaN,NaN,None,#TrumpTrain- @RealDonaldTrump is speaking out ...,1.109129e+18,2.648717e+07,LouDobbs,True,366.0,1901004.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,janetlchatham,9.0,3.0,3
3,1109131633603440640,242425184,Dominika Markova,dominikam42,Fri Mar 22 16:36:02 +0000 2019,RT @votevets: 1. John McCain was a war hero.\n...,0,17392,198,False,NaN,NaN,None,1. John McCain was a war hero.\n\n2. John McCa...,1.108447e+18,1.651334e+07,votevets,True,62408.0,158248.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,dominikam42,3.0,2.0,2
4,1109131633293037574,17200919,Michael Little,uzererr,Fri Mar 22 16:36:02 +0000 2019,RT @brithume: President Trump’s attacks on the...,0,1521,74,False,NaN,NaN,None,President Trump’s attacks on the late John McC...,1.109097e+18,1.120478e+08,brithume,True,9022.0,996715.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,uzererr,4.0,4.0,4


In [17]:
df_no_class = df_mccain[df_mccain['modularity_class'].isnull() == True]

In [18]:
df_no_class.sort_values('followers_count', ascending=False).head(20)

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq
926040,1107535220264316928,25979455,Daily Nation,dailynation,Mon Mar 18 06:52:28 +0000 2019,Trump renews attacks on McCain https://t.co/tJ...,7,1,1608426,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
452462,1108523134431432707,19923638,The Cut,TheCut,Thu Mar 21 00:18:05 +0000 2019,President Trump has made it clear that he woul...,3,0,1404940,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
787997,1108064755791454209,37874853,The Straits Times,STcom,Tue Mar 19 17:56:39 +0000 2019,RT @STForeignDesk: 'I was never a fan of John ...,0,1,1007766,True,NaN,NaN,None,"'I was never a fan of John McCain, and I never...",1.108065e+18,1.131353e+09,STForeignDesk,False,1.0,41206.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,1
127715,1108861856884670464,19621110,GOOD,good,Thu Mar 21 22:44:02 +0000 2019,When Trump attacks John McCain he’s attacking ...,2,1,888260,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
581130,1108464916980252682,18391051,Action News on 6abc,6abc,Wed Mar 20 20:26:45 +0000 2019,Trump on John McCain's funeral: 'I didn't get ...,19,5,857169,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
515522,1108493679713497089,184647530,Dose,dose,Wed Mar 20 22:21:02 +0000 2019,Sen. Johnny Isakson (R-Ga.) is pushing back ag...,6,1,836144,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
254079,1108740162077552640,14216661,National Post,nationalpost,Thu Mar 21 14:40:28 +0000 2019,Trump's ranting about John McCain again: Dead ...,1,1,804205,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
504760,1108498322552221696,15791186,KHOU 11 News Houston,KHOU,Wed Mar 20 22:39:29 +0000 2019,Trump ramps up attacks on late Sen. John McCai...,2,0,697267,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3
332212,1108647014928134149,15791186,KHOU 11 News Houston,KHOU,Thu Mar 21 08:30:20 +0000 2019,Trump ramps up attacks on late Sen. John McCai...,0,0,697266,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3
377509,1108568982494294016,15791186,KHOU 11 News Houston,KHOU,Thu Mar 21 03:20:16 +0000 2019,Trump ramps up attacks on late Sen. John McCai...,0,0,697266,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3


In [19]:
df_no_class.shape

(72912, 32)

In [20]:
df_mccain.shape

(1037841, 32)

In [21]:
509602-79489

430113

So it looks like the network communities just ignored those with very low frequency in the dataset, which is fine. If I ignore those without a modularity class, I end up with roughly 1.3 million observations (removing 307k). This should be fine for NLP analysis, so I'll just remove those

In [21]:
df_mccain = df_mccain[df_mccain['modularity_class'].isnull() == False]

In [22]:
df_mccain.shape
## And I'll pickle it

(964929, 32)

In [23]:
with open('/Users/robertpagano/metis_data/project_5/mccain_nlp_null_class_removed.pickle', 'wb') as to_write:
    pickle.dump(df_mccain, to_write)

Now filter for just communties with enough data, and find the top posters in each

2, 16 can be ignored

In [24]:
# list of communities to run NLP on
list_of_coms = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [25]:
df_mccain_coms = df_mccain[df_mccain['modularity_class'].isin(list_of_coms)]

In [26]:
df_mccain_coms.shape

(964892, 32)

In [27]:
# get counts of tweets by modularity class
df_mccain_coms.groupby(['modularity_class']).count()[['screen_name']]

,screen_name
modularity_class,
0.0,239728
1.0,252167
3.0,27621
4.0,47942
5.0,12178
6.0,11105
7.0,18715
8.0,31663
9.0,250012


 - append retweet_mod_class to all retweets?
 - Append pagerank to each tweet, so I can get most influential nodes by class
 - sort by follower count first, then come back and add pagerank, betweenness, etc.

In [74]:
# So basically I need to do a groupby, screenname, follower count, head(10), and functionize it

In [28]:
df_mccain_coms.columns

Index(['tweet_id', 'user_id', 'display_name', 'screen_name', 'tweet_date',
       'main_text', 'fav_count', 'rt_count', 'followers_count', 'is_verified',
       'reply_to_tweet_id', 'reply_to_user_id', 'reply_to_screen_name',
       'retweet_text', 'retweet_id', 'retweet_user_id', 'retweet_screen_name',
       'retweet_is_verified', 'retweet_fav_count', 'retweet_followers_count',
       'quoted_text', 'quoted_id', 'quoted_user_id', 'quoted_screen_name',
       'quoted_is_verified', 'quoted_fav_count', 'is_retweet', 'is_quote',
       'Label', 'modularity_class', 'Weighted Degree', 'screenname_freq'],
      dtype='object')

I'm going to add pagerank here, and then add it to the "raw" file, and then re-run all the separate communities things

In [30]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/mccain_pagerank.pickle', 'rb') as f:
    pr = pickle.load(f)

In [31]:
df_mccain_coms['PageRank'] = df_mccain_coms['screen_name'].map(pr)
df_mccain_coms.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq,PageRank
0,1109131638112432129,113817127,KDW,WYATT251,Fri Mar 22 16:36:03 +0000 2019,RT @thedilleyshow: Donald J. Trump is still Pr...,0,107,474,False,NaN,NaN,None,Donald J. Trump is still President.\n\nHillary...,1.109098e+18,1.085581e+18,thedilleyshow,False,299.0,8815.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,WYATT251,9.0,2.0,2,0.000002
1,1109131636262744066,4839872717,Lou Ella Howard,ellahoward271,Fri Mar 22 16:36:03 +0000 2019,RT @LindseyGrahamSC: Great read. Spot on.\n\n...,0,65,3601,False,NaN,NaN,None,Great read. Spot on.\n\nMuch more could be sa...,1.109117e+18,4.328953e+08,LindseyGrahamSC,True,181.0,756904.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,ellahoward271,9.0,2.0,6,0.000001
2,1109131635742564352,881906625799266304,God Is LOVE,janetlchatham,Fri Mar 22 16:36:03 +0000 2019,RT @LouDobbs: #TrumpTrain- @RealDonaldTrump is...,0,158,153,False,NaN,NaN,None,#TrumpTrain- @RealDonaldTrump is speaking out ...,1.109129e+18,2.648717e+07,LouDobbs,True,366.0,1901004.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,janetlchatham,9.0,3.0,3,0.000002
3,1109131633603440640,242425184,Dominika Markova,dominikam42,Fri Mar 22 16:36:02 +0000 2019,RT @votevets: 1. John McCain was a war hero.\n...,0,17392,198,False,NaN,NaN,None,1. John McCain was a war hero.\n\n2. John McCa...,1.108447e+18,1.651334e+07,votevets,True,62408.0,158248.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,dominikam42,3.0,2.0,2,0.000001
4,1109131633293037574,17200919,Michael Little,uzererr,Fri Mar 22 16:36:02 +0000 2019,RT @brithume: President Trump’s attacks on the...,0,1521,74,False,NaN,NaN,None,President Trump’s attacks on the late John McC...,1.109097e+18,1.120478e+08,brithume,True,9022.0,996715.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,uzererr,4.0,4.0,4,0.000002


In [32]:
df_coms_raw = df_mccain_coms.loc[:, ['screen_name', 'followers_count', 'modularity_class', 'main_text', 'retweet_text', 'tweet_date', 'PageRank', 'rt_count']]

In [33]:
df_com_0 = df_coms_raw[df_coms_raw['modularity_class'] == 0]
df_com_1 = df_coms_raw[df_coms_raw['modularity_class'] == 1]
df_com_3 = df_coms_raw[df_coms_raw['modularity_class'] == 3]
df_com_4 = df_coms_raw[df_coms_raw['modularity_class'] == 4]
df_com_5 = df_coms_raw[df_coms_raw['modularity_class'] == 5]
df_com_6 = df_coms_raw[df_coms_raw['modularity_class'] == 6]
df_com_7 = df_coms_raw[df_coms_raw['modularity_class'] == 7]
df_com_8 = df_coms_raw[df_coms_raw['modularity_class'] == 8]
df_com_9 = df_coms_raw[df_coms_raw['modularity_class'] == 9]
df_com_10 = df_coms_raw[df_coms_raw['modularity_class'] == 10]
df_com_11 = df_coms_raw[df_coms_raw['modularity_class'] == 11]
df_com_12 = df_coms_raw[df_coms_raw['modularity_class'] == 12]
df_com_13 = df_coms_raw[df_coms_raw['modularity_class'] == 13]
df_com_14 = df_coms_raw[df_coms_raw['modularity_class'] == 14]
df_com_15 = df_coms_raw[df_coms_raw['modularity_class'] == 15]


In [34]:
print(df_com_0.shape)
print(df_com_1.shape)
print(df_com_3.shape)
print(df_com_4.shape)
print(df_com_5.shape)
print(df_com_6.shape)
print(df_com_7.shape)
print(df_com_8.shape)
print(df_com_9.shape)
print(df_com_10.shape)
print(df_com_11.shape)
print(df_com_12.shape)
print(df_com_13.shape)
print(df_com_14.shape)
print(df_com_15.shape)


(239728, 8)
(252167, 8)
(27621, 8)
(47942, 8)
(12178, 8)
(11105, 8)
(18715, 8)
(31663, 8)
(250012, 8)
(11987, 8)
(11756, 8)
(15614, 8)
(8350, 8)
(6666, 8)
(19388, 8)


In [35]:
df_com_0 = df_com_0.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_1 = df_com_1.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_3 = df_com_3.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_4 = df_com_4.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_5 = df_com_5.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_6 = df_com_6.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_7 = df_com_7.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_8 = df_com_8.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_9 = df_com_9.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_10 = df_com_10.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_11 = df_com_11.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_12 = df_com_12.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_13 = df_com_13.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_14 = df_com_14.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_15 = df_com_15.sort_values(by=['PageRank', 'rt_count'], ascending=False)


In [36]:
df_com_0.head()

,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count
380860,funder,450281,0.0,RT @funder: 46 years ago Senator John McCain s...,46 years ago Senator John McCain stepped off a...,Thu Mar 21 03:10:00 +0000 2019,0.01512,15079
380871,funder,450281,0.0,RT @funder: .@LindseyGrahamSC-I would never le...,.@LindseyGrahamSC-I would never let someone at...,Thu Mar 21 03:09:58 +0000 2019,0.01512,8368
524272,funder,450309,0.0,I just wanna state for the record that Sen Lin...,NaN,Wed Mar 20 22:06:56 +0000 2019,0.01512,6379
22186,funder,450229,0.0,RT @funder: I just wanna state for the record ...,I just wanna state for the record that Sen Lin...,Fri Mar 22 13:41:03 +0000 2019,0.01512,6370
570671,funder,450318,0.0,Trump just attacked Senator McCain &amp; his f...,NaN,Wed Mar 20 20:48:11 +0000 2019,0.01512,5927


For the below - I want to create this with just a list of the most influential 20 in each category, for EDA purposes

In [37]:
df_com_0_top_20 = df_com_0.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_1_top_20 = df_com_1.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_3_top_20 = df_com_3.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_4_top_20 = df_com_4.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_5_top_20 = df_com_5.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_6_top_20 = df_com_6.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_7_top_20 = df_com_7.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_8_top_20 = df_com_8.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_9_top_20 = df_com_9.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_10_top_20 = df_com_10.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_11_top_20 = df_com_11.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_12_top_20 = df_com_12.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_13_top_20 = df_com_13.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_14_top_20 = df_com_14.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_15_top_20 = df_com_15.drop_duplicates(subset='screen_name', keep='first').head(20)


In [38]:
df_com_5_top_20

,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count
600646,bubbaprog,54064,5.0,Trump complains that John McCain did not come ...,NaN,Wed Mar 20 19:35:23 +0000 2019,0.001798,2494
599619,desusnice,906337,5.0,Tempted to make a ghost of John McCain twitter...,NaN,Wed Mar 20 19:39:42 +0000 2019,0.000966,999
171064,TheOnion,11077671,5.0,Trump Ramps Up Attacks On John McCain By Dragg...,NaN,Thu Mar 21 19:49:04 +0000 2019,0.000892,631
598706,mattyglesias,422281,5.0,"I’ve got to say, before asking Trump yet anoth...",NaN,Wed Mar 20 19:42:31 +0000 2019,0.000489,429
294340,MikeGravel,29741,5.0,"While I respect Bernie, I disagree with him st...",NaN,Thu Mar 21 12:26:04 +0000 2019,0.000451,418
893968,SeanMcElwee,91924,5.0,John McCain said in 2016: “I promise you that ...,NaN,Mon Mar 18 14:39:35 +0000 2019,0.000375,446
587850,david_j_roth,76241,5.0,"John McCain's ghost was very rude to me, so ru...",NaN,Wed Mar 20 20:10:07 +0000 2019,0.000295,196
735551,UrbanAchievr,19225,5.0,RT @MittRomney: I can’t understand why the Pre...,"I can’t understand why the President would, on...",Wed Mar 20 00:35:59 +0000 2019,0.000287,14507
632893,swin24,61280,5.0,RT @THEHermanCain: Why is it that John McCain ...,Why is it that John McCain can't be criticized...,Wed Mar 20 15:49:05 +0000 2019,0.000268,3974
596714,crushingbort,66531,5.0,John McCain has not tweeted since August 2018,NaN,Wed Mar 20 19:47:35 +0000 2019,0.000239,193


Pickling all of my community partitioned data, will begin work on topic modeling in new notebook

In [40]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_0.pickle', 'wb') as to_write:
    pickle.dump(df_com_0, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_1.pickle', 'wb') as to_write:
    pickle.dump(df_com_1, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_3.pickle', 'wb') as to_write:
    pickle.dump(df_com_3, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_4.pickle', 'wb') as to_write:
    pickle.dump(df_com_4, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_5.pickle', 'wb') as to_write:
    pickle.dump(df_com_5, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_6.pickle', 'wb') as to_write:
    pickle.dump(df_com_6, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_7.pickle', 'wb') as to_write:
    pickle.dump(df_com_7, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_8.pickle', 'wb') as to_write:
    pickle.dump(df_com_8, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_9.pickle', 'wb') as to_write:
    pickle.dump(df_com_9, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_10.pickle', 'wb') as to_write:
    pickle.dump(df_com_10, to_write)
     
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_11.pickle', 'wb') as to_write:
    pickle.dump(df_com_11, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_12.pickle', 'wb') as to_write:
    pickle.dump(df_com_12, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_13.pickle', 'wb') as to_write:
    pickle.dump(df_com_13, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_14.pickle', 'wb') as to_write:
    pickle.dump(df_com_14, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/mccain/community_pickles_pre_nlp/df_15.pickle', 'wb') as to_write:
    pickle.dump(df_com_15, to_write)
    